In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
# Libraries
import tensorflow as tf
import pathlib
from tqdm import tqdm
import matplotlib.pylab as plt

In [4]:
mnist = tf.keras.datasets.mnist  

(training_images, training_labels), (val_images, val_labels) = mnist.load_data()    # Load MNIST dataset

# Reshape and alter the pixel values of the data
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
val_images=val_images.reshape(10000, 28, 28, 1)
val_images=val_images/255.0

# Alter the training set images
for train in range(len(training_images)):
  for row in range(28):
    for cell in range(28):
      if training_images[train][row][cell][0] != 0:
        training_images[train][row][cell][0] = 1
        
# alter the validation set images
for val in range(len(val_images)):
  for row in range(28):
    for cell in range(28):
      if val_images[val][row][cell][0] != 0:
        val_images[val][row][cell][0] = 1

11493376/11490434 [==============================] - 0s 0us/step


In [23]:

# Build the model
model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')                           
])

# define and instantiate your custom Callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if( logs.get('val_accuracy') > 0.99 or epoch >= 10):
      self.model.stop_training = True
callbacks = myCallback()

# Compile the model
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Fit the model
model.fit(training_images, training_labels, validation_data=(val_images, val_labels), epochs=20, callbacks=[callbacks])



Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


1875/1875 [==============================] - 6s 3ms/step - loss: 0.2825 - accuracy: 0.9137 - val_loss: 0.1394 - val_accuracy: 0.9586
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1317 - accuracy: 0.9595 - val_loss: 0.1169 - val_accuracy: 0.9646
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0970 - accuracy: 0.9699 - val_loss: 0.1081 - val_accuracy: 0.9673
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0800 - accuracy: 0.9749 - val_loss: 0.0993 - val_accuracy: 0.9712
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0665 - accuracy: 0.9777 - val_loss: 0.1102 - val_accuracy: 0.9673
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0553 - accuracy: 0.9822 - val_loss: 0.0945 - val_accuracy: 0.9734
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0484 - accuracy: 0.9841 - val_loss: 0.0962 - val_accuracy: 0.9747
Epo

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f76f51c7950>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


In [8]:
# Save the model
MNIST_MODEL = "exp_saved_model"
tf.saved_model.save(model, MNIST_MODEL)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: exp_saved_model/assets


In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model(MNIST_MODEL)

# Uncomment to optimize with quantization aware training
'''converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_data_gen():
  for input_value, _ in test_batches.take(80):
    yield [input_value]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
'''

tflite_model = converter.convert()    # Convert the model to a tensorflow lite model

# Create the path to save the model
tflite_models_dir = pathlib.Path("/tmp/")
tflite_model_file = tflite_models_dir/'model1.tflite'

# Write bytes
tflite_model_file.write_bytes(tflite_model)

475300